Autoreload notebook

In [1]:
%load_ext autoreload
%autoreload 2

Imort libraries

In [1]:
import os
import warnings
import sys
import datetime

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import pickle

Import modules

In [2]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from plot import Plot
from preprocess import Preprocess
from load_data import LoadData

Instantiate modules

In [3]:
loader = LoadData()

>> Get Data

In [4]:
# Get URL from DVC
path = 'data/train_processed.csv'
repo = 'https://github.com/jedisam/Sales-prediction'
version = '06bfe0a6a4297a694b365bfdaa30e2184a10f38c'

Load data using dvc

In [5]:
# Load data from dvc using the dvc.api.Dataset class
data_url = dvc.api.get_url(
    path=path,
    repo=repo,
    rev=version
)

In [6]:
data = loader.read_csv('data_url')
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.set_index('Date', inplace=True)

In [7]:
data.head()

,Store,DayOfWeek,Sales,Open,Promo,StateHoliday,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,...,Promo2SinceYear,Year,Month,Day,weekday,weekofyear,weekend,PromoInterval,Assortment,StoreType
Date,,,,,,,,,,,,,,,,,,,,,
2015-07-31,1,4,5263.0,1,1,0,1,1270.0,9.0,2008.0,...,2012.0,2015,7,31,4,31,0,1,0,2
2015-07-31,2,4,6064.0,1,1,0,1,570.0,11.0,2007.0,...,2010.0,2015,7,31,4,31,0,1,0,0
2015-07-31,3,4,8314.0,1,1,0,1,14130.0,12.0,2006.0,...,2011.0,2015,7,31,4,31,0,1,0,0
2015-07-31,4,4,13995.0,1,1,0,1,620.0,9.0,2009.0,...,2012.0,2015,7,31,4,31,0,1,2,2
2015-07-31,5,4,4822.0,1,1,0,1,29910.0,4.0,2015.0,...,2012.0,2015,7,31,4,31,0,1,0,0


In [8]:
data.drop(['StateHoliday'], axis=1, inplace=True)

Drop rows that are closed (close=0) & sales=0

In [9]:
data = data[data['Open'] == 1]
data = data[data['Sales'] > 0.0]

>> Split the data

In [10]:
X = data.drop('Sales',axis=1)
y = data['Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

>> Use Sklearn Pipeline

In [12]:
data_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])

pipeline = Pipeline(steps = [
               ('preprocessor', data_transformer)
              ,('regressor',RandomForestRegressor(max_depth=64, random_state=0))
           ])

In [13]:
rf_model = pipeline.fit(X_train, y_train)
print(rf_model)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('imputer', SimpleImputer()),
                                 ('scaler', StandardScaler())])),
                ('regressor',
                 RandomForestRegressor(max_depth=64, random_state=0))])


In [14]:
rf_model.score(X_test, y_test)

0.9160431993111918

In [20]:
# Save model based on time st
rf_model_path = '../models/' + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '.pkl'
pickle.dump(rf_model, open(rf_model_path, 'wb'))